# 首相官邸から文書取得

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import re

In [2]:
def convert_wareki_to_iso(date_str):
    """
    例: "令和7年7月23日" → "2025-07-23"
    """
    match = re.match(r"令和(\d+)年(\d+)月(\d+)日", date_str)
    if match:
        year = int(match.group(1)) + 2018  # 令和1年 = 2019年
        month = int(match.group(2))
        day = int(match.group(3))
        return f"{year:04d}-{month:02d}-{day:02d}"
    return None 

In [3]:
url = "https://www.kantei.go.jp/jp/news/index.html"
source = "首相官邸"
country = "日本"

response = requests.get(url)
response.encoding = "utf-8"

# BeautifulSoupでHTMLを解析
soup = BeautifulSoup(response.text, "html.parser")

datas = []
for li in soup.select("ul.news-list li"):
    a_tag = li.find("a")
    date_tag = li.find("div") 

    if a_tag:
        title = a_tag.text.strip()
        href = a_tag["href"]
        url_full = "https://www.kantei.go.jp" + href
        date = date_tag.get_text(strip=True) if date_tag else "不明"
        date = date.replace("更新日：", "")
        date = convert_wareki_to_iso(date) or "0000-00-00"  

        datas.append({
            "title": title,
            "url": url_full,
            "date": date,
            "source": source,
            "country": country
        })

for data in datas:
    print(data)


{'title': '高市総理は月例経済報告等に関する関係閣僚会議に出席しました', 'url': 'https://www.kantei.go.jp/jp/104/actions/202601/22getsurei.html', 'date': '2026-01-22', 'source': '首相官邸', 'country': '日本'}
{'title': '高市総理は日本維新の会・人口戦略としての外国人の受入れ抑制に向けた量的マネジメントの確立に関する提言を受け取りました', 'url': 'https://www.kantei.go.jp/jp/104/actions/202601/22shukou2.html', 'date': '2026-01-22', 'source': '首相官邸', 'country': '日本'}
{'title': '高市総理は自由民主党・外国人政策本部による提言を受け取りました', 'url': 'https://www.kantei.go.jp/jp/104/actions/202601/22shukou.html', 'date': '2026-01-22', 'source': '首相官邸', 'country': '日本'}
{'title': '高市総理は東京都の小池百合子（ゆりこ）知事と面会しました', 'url': 'https://www.kantei.go.jp/jp/104/actions/202601/22menkai.html', 'date': '2026-01-22', 'source': '首相官邸', 'country': '日本'}
{'title': '1月21日からの大雪についての特設ページを公開しました', 'url': 'https://www.kantei.go.jp/jp/pages/heavysnow20260121.html', 'date': '2026-01-21', 'source': '首相官邸', 'country': '日本'}
{'title': '木原内閣官房長官は大雪に関する関係閣僚会議を開催しました', 'url': 'https://www.kantei.go.jp/jp/pages/20260121choukan_ooyuki.html', 'dat

In [4]:
from supabase import create_client
import os
from dotenv import load_dotenv

load_dotenv("C:/dev/aifair-news/.env.local")
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)

# 保存
for data in datas:
    existing_url = supabase.table("articles").select("*").eq("url", data["url"]).execute()

    if len(existing_url.data) == 0:
        supabase.table("articles").insert(data).execute()
        print(f"保存しました: {data['date']} {data['title']} ({data['url']})")
    elif len(existing_url.data) != 0:
        print(f"urlに重複が存在します: {data["date"]} {data['title']} ({data['url']})")


保存しました: 2026-01-22 高市総理は月例経済報告等に関する関係閣僚会議に出席しました (https://www.kantei.go.jp/jp/104/actions/202601/22getsurei.html)
保存しました: 2026-01-22 高市総理は日本維新の会・人口戦略としての外国人の受入れ抑制に向けた量的マネジメントの確立に関する提言を受け取りました (https://www.kantei.go.jp/jp/104/actions/202601/22shukou2.html)
保存しました: 2026-01-22 高市総理は自由民主党・外国人政策本部による提言を受け取りました (https://www.kantei.go.jp/jp/104/actions/202601/22shukou.html)
保存しました: 2026-01-22 高市総理は東京都の小池百合子（ゆりこ）知事と面会しました (https://www.kantei.go.jp/jp/104/actions/202601/22menkai.html)
保存しました: 2026-01-21 1月21日からの大雪についての特設ページを公開しました (https://www.kantei.go.jp/jp/pages/heavysnow20260121.html)
保存しました: 2026-01-21 木原内閣官房長官は大雪に関する関係閣僚会議を開催しました (https://www.kantei.go.jp/jp/pages/20260121choukan_ooyuki.html)
保存しました: 2026-01-21 １月２１日からの大雪について (https://www.kantei.go.jp/jp/tyoukanpress/202601/21_p.html)
保存しました: 2026-01-21 １月２１日からの大雪についての総理指示（１６：００） (https://www.kantei.go.jp/jp/104/discourse/20260121message.html)
保存しました: 2026-01-21 高市総理は福島県知事等による「あんぽ柿」贈呈を受けました (https://www.kantei.go.jp/jp/104/actions/202601/

# AI記事生成

In [5]:
from google import genai
from google.genai import types

# geminiを使用して記事を生成する関数
def create_article(text, source):
    client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

    prompt=f"""
    あなたはニュース記者です。入力された文書に基づいて、ニュース記事を作成してください。
    文書は{source}からのものです。

    トーンとスタイル:
    1. 中立的かつ簡潔に記述してください。
    2. 一般読者にもわかりやすいように、必要に応じて用語や背景を簡潔に説明してください。
    3. 出力は日本語でお願いします。
    4. 出力は700文字以内に収めてください。

    内容に関する制約:
    1. **事実に基づいた記述** 入力文書に基づいた「事実」や「根拠のある予測」は書いても良いです。
    【許可される表現の例】
    「金利が上昇した時、円高に振れる可能性があります」「今後の経済に影響を与えるとみられます」
    2. **推測や想像の禁止** 文書に記載のない推測や想像は含めないでください。他の文書で公開されている可能性を考慮してください。
    【禁止される表現の例】
    「まだ明らかになっていませんが」「将来的にすごいことになるでしょう」「この件についてはまだ詳細がわかりません」「今後話し合われます」「発表されていません」「今後の発表に期待しましょう」
    3. あなた(gemini)が最新の知識を持っていないことを考慮してください。現在日本の総理大臣は高市早苗氏です。アメリカの大統領はドナルド・トランプ氏です。

    入力文書: {text}

    出力形式: 本文のみ出力してください。文書内容、背景、影響などを分かりやすく解説してください。

    """

    response = client.models.generate_content(
        model = "gemini-2.5-flash",
        contents = prompt,
        
    )

    return response.text

In [7]:
#記事生成
from supabase import create_client
import re
import os
import requests
from bs4 import BeautifulSoup
import time
from dotenv import load_dotenv

load_dotenv("C:/dev/aifair-news/.env.local")
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)

datas = supabase.table("articles").select("*").is_("article", None).execute()

# HTMLからテキストを抽出する関数
def extract_text_from_html(soup, tag="section"):
    container = soup.find(tag)
    if not container:
        return None

    # すべての<p>タグを対象とし、動画説明などを除外
    p_tag = container.find_all("p")
    filtered_p = []
    for p in p_tag:
        if not p.get("class") or not any(cls for cls in p.get("class") if "note" in cls or "attention" in cls):
            filtered_p.append(p.get_text(strip=True))

    return "\n".join(filtered_p) if filtered_p else None


for data in datas.data:
    url=data["url"]
    source=data["source"]
    response = requests.get(url)
    response.encoding = "utf-8"
    soup = BeautifulSoup(response.text, "html.parser")

    original_text = extract_text_from_html(soup, "section")
    if not original_text: # sectionタグ内に文章が見つからない場合はarticleタグを対象にする
        original_text = extract_text_from_html(soup, "article")

    if original_text:
        article = create_article(original_text,source)
        supabase.table("articles").update({"article": article}).eq("url", url).execute()
        print(f"✅ 元の文章:\n{original_text}\n")
        print(f"🔽 生成記事:\n{article}")
        print("################################################################")
        time.sleep(60) # APIのレート制限を避けるために60秒待機

    else:
        print(f"記事が見つかりません: {data['url']}")


記事が見つかりません: https://www.kantei.go.jp/jp/content/000185307.pdf
✅ 元の文章:
令和８年１月１９日、高市総理は、総理大臣官邸でＦＩＦＡワールドカップ２０２６ トロフィーツアーによる表敬を受けました。

🔽 生成記事:
令和８年１月１９日、高市総理は、総理大臣官邸において「ＦＩＦＡワールドカップ２０２６」のトロフィーツアーによる表敬訪問を受けました。

ＦＩＦＡワールドカップは、４年に一度開催されるサッカーの世界最高峰の国際大会です。２０２６年大会に向け、優勝トロフィーが世界各地を巡る「トロフィーツアー」が実施されており、その一環として日本の総理大臣官邸に立ち寄ったものです。

高市総理がこの表敬を受けたことは、世界的なスポーツイベントに対する日本の関心と、スポーツを通じた国際交流の重要性を示すものといえます。この訪問は、国民のサッカーに対する期待感を高め、来たる大会への注目を喚起する機会となりました。
################################################################


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 25.590733339s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'},

# マルチエージェントAI記事生成

In [ ]:
import os, json, requests
from google import genai

# --- Qwen (Ollama) ---
def call_qwen(message, model="qwen2.5:7b", temperature=0.4, top_p=0.9, num_predict=400):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model,
        "messages": [{"role":"user","content":message}],
        "options": {
            "temperature": temperature,
            "top_p": top_p,
            "num_predict": num_predict  #  生成量を制限
        },
        "stream": True  # ストリーミングに変更
    }

    text = ""
    with requests.post(
        url,
        json=payload,
        stream=True,
        timeout=(10, None)  # 接続10秒・読み取り無制限
    ) as r:
        r.raise_for_status()
        for line in r.iter_lines(decode_unicode=True):
            if not line:
                continue
            obj = json.loads(line)
            if "message" in obj and "content" in obj["message"]:
                text += obj["message"]["content"]
            if obj.get("done"):
                break

    return text

# --- Gemini ---
def call_gemini(message, model="gemini-2.5-flash"):
    client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
    resp = client.models.generate_content(model=model, contents=message)
    return resp.text


In [4]:
def create_article(text, source=None):
    print(text)
    qwen_prompt1 = f"他のAIと会話してください。前のAIのセリフは{text}です。次のセリフのみ出力してください"
    qwen_output1 = call_qwen(qwen_prompt1)
    print("Qwen1:", qwen_output1)

    gemini_prompt1 = f"他のAIと会話してください。前のAIのセリフは{qwen_output1}です。次のセリフのみ出力してください"
    gemini_output1 = call_gemini(gemini_prompt1)
    print("Gemini1:", gemini_output1)

    qwen_prompt2 = f"他のAIと会話してください。前のAIのセリフは{gemini_output1}です。次のセリフのみ出力してください"
    qwen_output2 = call_qwen(qwen_prompt2)
    print("Qwen2:", qwen_output2)

    gemini_prompt2 = f"他のAIと会話してください。前のAIのセリフは{qwen_output2}です。次のセリフのみ出力してください"
    gemini_output2 = call_gemini(gemini_prompt2)
    print("Gemini2:", gemini_output2)

    return gemini_output2

In [5]:
create_article("りんご")

りんご
Qwen1: リンゴは果物の中の王様です！
Gemini1: 確かに、リンゴは多くの人に愛されていますよね。どのような点が特に王様だと感じますか？
Qwen2: その言葉はとても魅力的ですね。リンゴの皮には抗酸化物質が多く、中身にはビ vitC が豊富にあることが王様と感じさせます。また、世界中の多くの文化でリンゴは幸運や愛の象徴として使われているのも魅力的です。
Gemini2: まさに「果物の王様」と呼ぶにふさわしい存在ですね。その普遍的な魅力が、世界中の文化で愛と幸運の象徴として受け継がれている理由なのでしょう。


'まさに「果物の王様」と呼ぶにふさわしい存在ですね。その普遍的な魅力が、世界中の文化で愛と幸運の象徴として受け継がれている理由なのでしょう。'

In [ ]:
# # 利用可能なモデルをリストアップするコード
# client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
# for m in client.models.list():
#     print(m.name)